In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

In [2]:
train=pd.read_csv('./Dataset/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('./Dataset/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [3]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def build_model(input_dim):
    # Define a sequential model
    model = Sequential()

    # Add the layers
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Build the model
model = build_model(X_train.shape[1])

# Fit the model
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    epochs=100, 
    batch_size=32, 
    callbacks=[early_stopping]
)

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)

print(f'Train loss: {train_loss:.4f}, Train accuracy: {train_accuracy:.4f}')
print(f'Validation loss: {val_loss:.4f}, Validation accuracy: {val_accuracy:.4f}')


Epoch 1/100
10/10 [==============================] - 1s 30ms/step - loss: 0.6855 - accuracy: 0.6159 - val_loss: 0.5554 - val_accuracy: 0.7671
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5761 - accuracy: 0.7509 - val_loss: 0.4899 - val_accuracy: 0.8082
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.5179 - accuracy: 0.7924 - val_loss: 0.4549 - val_accuracy: 0.8082
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4763 - accuracy: 0.8097 - val_loss: 0.4243 - val_accuracy: 0.8356
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4500 - accuracy: 0.8235 - val_loss: 0.4082 - val_accuracy: 0.8082
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4217 - accuracy: 0.8339 - val_loss: 0.4144 - val_accuracy: 0.8356
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.4015 - accuracy: 0.8443 - val_loss: 0.4135 - val_accuracy: 0.8356
Epoch 8/100


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# DNN 모델 정의
class DNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DNN, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        layers = [nn.Linear(input_size, hidden_sizes[0])]
        for i in range(1, len(hidden_sizes)):
            layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
        self.hidden_layers = nn.ModuleList(layers)

        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = nn.ReLU()(layer(x))
        x = self.output_layer(x)
        return x

def train_dnn_with_kfold(X_train, y_train, X_test, y_test, k):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # DNN 모델 초기화 및 학습
        input_size = X_train_fold.shape[1]
        hidden_sizes = [64, 32, 16, 8]
        hidden_sizes = [64, 32, 16, 8]
        output_size = 1
        model = DNN(input_size, hidden_sizes, output_size)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_fold.values, dtype=torch.float32).view(-1, 1).to(device)

        num_epochs = 1000
        for epoch in range(num_epochs):
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 검증 데이터에 대한 예측 및 평가
        X_val_tensor = torch.tensor(X_val_fold.values, dtype=torch.float32).to(device)
        with torch.no_grad():
            model.eval()
            y_val_pred_tensor = model(X_val_tensor)
            y_val_pred = (y_val_pred_tensor >= 0.5).view(-1).cpu().numpy()

        # 평가 지표 계산
        accuracy = accuracy_score(y_val_fold, y_val_pred)
        precision = precision_score(y_val_fold, y_val_pred)
        recall = recall_score(y_val_fold, y_val_pred)
        f1 = f1_score(y_val_fold, y_val_pred)
        conf_matrix = confusion_matrix(y_val_fold, y_val_pred)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"{fold_idx}번째 Fold")
        print("평가 지표")
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 Score:", f1)
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 F1 스코어를 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 전체 교차 검증 결과 출력
    print("전체 교차 검증 결과")
    print("평균 Accuracy:", sum(accuracy_list) / len(accuracy_list))
    print("평균 Precision:", sum(precision_list) / len(precision_list))
    print("평균 Recall:", sum(recall_list) / len(recall_list))
    print("평균 F1 Score:", sum(f1_score_list) / len(f1_score_list))

    # 가장 좋은 F1 스코어를 가진 모델로 최종 예측 수행
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    with torch.no_grad():
        best_model.eval()
        y_test_pred_tensor = best_model(X_test_tensor)
        y_test_pred = (y_test_pred_tensor >= 0.5).view(-1).cpu().numpy()

    # 테스트 데이터에 대한 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_test_pred)
    precision_final = precision_score(y_test, y_test_pred)
    recall_final = recall_score(y_test, y_test_pred)
    f1_final = f1_score(y_test, y_test_pred)
    conf_matrix_final = confusion_matrix(y_test, y_test_pred)

    print("테스트 데이터 평가 결과")
    print("Accuracy:", accuracy_final)
    print("Precision:", precision_final)
    print("Recall:", recall_final)
    print("F1 Score:", f1_final)
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return best_model


In [7]:
train_dnn_with_kfold(X_train, y_train, X_test, y_test, k=7)

1번째 Fold
평가 지표
Accuracy: 0.8095238095238095
Precision: 0.6666666666666666
Recall: 0.5454545454545454
F1 Score: 0.6
Confusion Matrix:
[[28  3]
 [ 5  6]]
------------------------------
2번째 Fold
평가 지표
Accuracy: 0.7380952380952381
Precision: 0.5
Recall: 0.45454545454545453
F1 Score: 0.47619047619047616
Confusion Matrix:
[[26  5]
 [ 6  5]]
------------------------------
3번째 Fold
평가 지표
Accuracy: 0.8536585365853658
Precision: 0.75
Recall: 0.6
F1 Score: 0.6666666666666665
Confusion Matrix:
[[29  2]
 [ 4  6]]
------------------------------
4번째 Fold
평가 지표
Accuracy: 0.7317073170731707
Precision: 0.5
Recall: 0.5454545454545454
F1 Score: 0.5217391304347826
Confusion Matrix:
[[24  6]
 [ 5  6]]
------------------------------
5번째 Fold
평가 지표
Accuracy: 0.8536585365853658
Precision: 0.8571428571428571
Recall: 0.5454545454545454
F1 Score: 0.6666666666666665
Confusion Matrix:
[[29  1]
 [ 5  6]]
------------------------------
6번째 Fold
평가 지표
Accuracy: 0.8292682926829268
Precision: 0.7
Recall: 0.6363636363636

DNN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=49, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Linear(in_features=16, out_features=8, bias=True)
  )
  (output_layer): Linear(in_features=8, out_features=1, bias=True)
)